<a href="https://colab.research.google.com/github/MysticalHeat/python_labs/blob/main/%D0%9B%D0%B0%D0%B1_5_%D0%9D%D0%B0%D1%81%D1%82%D1%80%D0%B9%D0%BA%D0%B8_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Авторские права принадлежат авторам TensorFlow, 2020 г.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Знакомство с тюнером Keras

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/keras/keras_tuner"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/keras/keras_tuner.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/keras/keras_tuner.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/keras/keras_tuner.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

## Обзор

Keras Tuner — это библиотека, которая помогает вам выбрать оптимальный набор гиперпараметров для вашей программы TensorFlow. Процесс выбора правильного набора гиперпараметров для вашего приложения машинного обучения (ML) называется *настройкой гиперпараметров* или *гипернастройкой*.

Гиперпараметры — это переменные, которые управляют процессом обучения и топологией модели ML. Эти переменные остаются постоянными в течение всего процесса обучения и напрямую влияют на производительность вашей программы ML. Гиперпараметры бывают двух типов:
1. **Гиперпараметры модели**, которые влияют на выбор модели, например количество и ширина скрытых слоев.
2. **Гиперпараметры алгоритма**, которые влияют на скорость и качество алгоритма обучения, такие как скорость обучения для стохастического градиентного спуска (SGD) и количество ближайших соседей для классификатора k ближайших соседей (KNN).

В этом руководстве вы будете использовать Keras Tuner для выполнения гипернастройки приложения классификации изображений.

## Настраивать

In [2]:
import tensorflow as tf
from tensorflow import keras

Установите и импортируйте Keras Tuner.

In [3]:
!pip install -q -U keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 5.3 MB/s eta 0:00:00


In [4]:
import keras_tuner as kt

## Загрузите и подготовьте набор данных

В этом уроке вы будете использовать Keras Tuner, чтобы найти лучшие гиперпараметры для модели машинного обучения, которая классифицирует изображения одежды из [набора данных Fashion MNIST] (https://github.com/zalandoresearch/fashion-mnist).

Загрузите данные.

In [5]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [6]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

## Определим модель

Когда вы создаете модель для гипернастройки, вы также определяете пространство поиска гиперпараметров в дополнение к архитектуре модели. Модель, которую вы настраиваете для гипернастройки, называется *гипермоделью*.

Вы можете определить гипермодель двумя способами:

* С помощью функции построения моделей
* Путем создания подкласса класса HyperModel API Keras Tuner.

Вы также можете использовать два предопределенных класса [HyperModel](https://keras.io/api/keras_tuner/hypermodels/) — [HyperXception](https://keras.io/api/keras_tuner/hypermodels/hyper_xception/). и [HyperResNet](https://keras.io/api/keras_tuner/hypermodels/hyper_resnet/) для приложений компьютерного зрения.

В этом руководстве вы используете функцию построителя моделей для определения модели классификации изображений. Функция построителя моделей возвращает скомпилированную модель и использует встроенные гиперпараметры для гипернастройки модели.

In [7]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(keras.layers.Dense(units=hp_units, activation='relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

## Создайте экземпляр тюнера и выполните гипернастройку

Создайте экземпляр тюнера для выполнения гипернастройки. Keras Tuner имеет четыре доступных тюнера: «RandomSearch», «Hyperband», «BayesianOptimization» и «Sklearn». В этом руководстве вы используете тюнер [Hyperband](https://arxiv.org/pdf/1603.06560.pdf).

Чтобы создать экземпляр тюнера Hyperband, вы должны указать гипермодель, «цель» для оптимизации и максимальное количество эпох для обучения («max_epochs»).

In [8]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Алгоритм настройки Hyperband использует адаптивное распределение ресурсов и раннюю остановку для быстрого перехода к высокопроизводительной модели. Это делается с помощью кронштейна в стиле спортивного чемпионата. Алгоритм обучает большое количество моделей в течение нескольких эпох и переносит в следующий раунд только наиболее эффективную половину моделей. Hyperband определяет количество моделей для обучения в группе, вычисляя 1 + log<sub>`factor`</sub>(`max_epochs`) и округляя его до ближайшего целого числа.

Создайте обратный вызов, чтобы прекратить обучение досрочно после достижения определенного значения потери при проверке.

In [9]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

Запустите поиск гиперпараметров. Аргументы метода поиска такие же, как и для tf.keras.model.fit, в дополнение к обратному вызову выше..

In [10]:
tuner.search(img_train, label_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 30 Complete [00h 01m 01s]
val_accuracy: 0.8513333201408386

Best val_accuracy So Far: 0.8933333158493042
Total elapsed time: 00h 13m 10s

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 288 and the optimal learning rate for the optimizer
is 0.001.



In [ ]:
best_hps

## Обучите модель

Найдите оптимальное количество эпох для обучения модели с помощью гиперпараметров, полученных в результате поиска.

In [11]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(img_train, label_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - accuracy: 0.7766 - loss: 0.6340 - val_accuracy: 0.8481 - val_loss: 0.4122
Epoch 2/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8624 - loss: 0.3803 - val_accuracy: 0.8635 - val_loss: 0.3791
Epoch 3/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step - accuracy: 0.8752 - loss: 0.3390 - val_accuracy: 0.8689 - val_loss: 0.3757
Epoch 4/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.8858 - loss: 0.3134 - val_accuracy: 0.8817 - val_loss: 0.3313
Epoch 5/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8931 - loss: 0.2907 - val_accuracy: 0.8827 - val_loss: 0.3259
Epoch 6/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8968 - loss: 0.2766 - val_accuracy: 0.8802 - val_loss: 0.3327
Epoch 7/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - accuracy: 0.9049 - loss: 0.2561 - val_accuracy: 0.8884 - val_loss: 0.3167
Epoch 8/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.9073 - loss: 0.2457

Восстановите гипермодель и обучите ее оптимальному количеству эпох сверху.

In [12]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(img_train, label_train, epochs=best_epoch, validation_split=0.2)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/21
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.7768 - loss: 0.6301 - val_accuracy: 0.8478 - val_loss: 0.4172
Epoch 2/21
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8593 - loss: 0.3850 - val_accuracy: 0.8552 - val_loss: 0.3968
Epoch 3/21
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.8763 - loss: 0.3388 - val_accuracy: 0.8740 - val_loss: 0.3547
Epoch 4/21
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8864 - loss: 0.3046 - val_accuracy: 0.8716 - val_loss: 0.3651
Epoch 5/21
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8923 - loss: 0.2897 - val_accuracy: 0.8834 - val_loss: 0.3279
Epoch 6/21
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9000 - loss: 0.2731 - val_accuracy: 0.8696 - val_loss: 0.3590
Epoch 7/21
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9031 - loss: 0.2625 - val_accuracy: 0.8871 - val_loss: 0.3265
Epoch 8/21
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9100 - loss: 0.2480 - 

Чтобы закончить это руководство, оцените гипермодель на тестовых данных.

In [13]:
eval_result = hypermodel.evaluate(img_test, label_test)
print("[test loss, test accuracy]:", eval_result)

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.8866 - loss: 0.3869
[test loss, test accuracy]: [0.38557541370391846, 0.8852999806404114]


Каталог `my_dir/intro_to_kt` содержит подробные журналы и контрольные точки для каждой пробной версии (конфигурации модели), запускаемой во время поиска гиперпараметров. Если вы повторно запустите поиск гиперпараметров, Keras Tuner будет использовать существующее состояние из этих журналов для возобновления поиска. Чтобы отключить это поведение, передайте дополнительный аргумент overwrite=True при создании экземпляра тюнера.

## Краткое содержание

В этом уроке вы узнали, как использовать Keras Tuner для настройки гиперпараметров модели. Чтобы узнать больше о Keras Tuner, посетите эти дополнительные ресурсы:

* [Keras Tuner в блоге TensorFlow](https://blog.tensorflow.org/2020/01/hyperparameter-tuning-with-keras-tuner.html)
* [Веб-сайт Keras Tuner](https://keras-team.github.io/keras-tuner/)

Также посетите [Панель управления HParams](https://www.tensorflow.org/tensorboard/hyperparameter_tuning_with_hparams) в TensorBoard, чтобы в интерактивном режиме настроить гиперпараметры вашей модели.